 计算机视觉（24春）作业1-5（30分）
---

## 题目: 相机标定实验
- 有一个3D空间中的棋盘格平面，大小为8x8，边长为1个单位长度。假设棋盘格平面位于世界坐标系中`z=0`的平面。
- 同时给你提供一个虚拟相机（理想相机，参数仅包含焦距和主点坐标），能够获得3维棋盘格的二维图像，大小为`400x400`，你可以移动它来获得不同的观察角度下的棋盘格平面的图像（简化为棋盘格角点图像）。
- 请你自行移动相机，获得多个不同角度下的棋盘格平面的图像，利用这些点的坐标，实现相机的标定。

---

### 环境配置见作业HW1-4

---

## 相机模型和3D棋盘格模型

- 以下的代码块里包含一个实现好的相机模型和3D棋盘格模型，你可以直接运行它们来实现相机标定。


In [100]:
import numpy as np
import cv2

"""相机模型"""
class CameraModel(object):
    dft_configs = {
        "fx": 200, # focal length
        "fy": 200,
        "cx": 200, # principal point
        "cy": 200,
        "yaw": 0, # degrees
        "pitch": 0, # degrees
        "roll": 0, # degrees
        "x": 0, # translation
        "y": 0,
        "z": 10,
    }
    def __init__(self, configs={}) -> None:
        self.configs = {**self.dft_configs, **configs}
        self.fx = self.configs["fx"]
        self.fy = self.configs["fy"]
        self.cx = self.configs["cx"]
        self.cy = self.configs["cy"]
        self.K = np.array([[self.fx, 0, self.cx], [0, self.fy, self.cy], [0, 0, 1]]) #

        self.yaw = self.configs["yaw"]
        self.pitch = self.configs["pitch"]
        self.roll = self.configs["roll"]
        self.R = self._get_rotation_matrix(self.yaw, self.pitch, self.roll)

        self.x = self.configs["x"]
        self.y = self.configs["y"]
        self.z = self.configs["z"]
        self.T = np.array([self.x, self.y, self.z]) # 平移向量

    def show_K(self):
        print(f"K: {self.K}")
    
    def show_R(self):
        print(f"R: {self.R}")

    def show_T(self):
        print(f"T: {self.T}")

    def _get_rotation_matrix(self, yaw, pitch, roll):
        yaw = np.deg2rad(yaw) #x * pi / 180
        pitch = np.deg2rad(pitch)
        roll = np.deg2rad(roll)
        Rx = np.array([[1, 0, 0], [0, np.cos(pitch), -np.sin(pitch)], [0, np.sin(pitch), np.cos(pitch)]])
        Ry = np.array([[np.cos(yaw), 0, np.sin(yaw)], [0, 1, 0], [-np.sin(yaw), 0, np.cos(yaw)]])
        Rz = np.array([[np.cos(roll), -np.sin(roll), 0], [np.sin(roll), np.cos(roll), 0], [0, 0, 1]])
        return Rz @ Ry @ Rx

    def project(self, points3D):
        """ Project 3D points to 2D
        Args:
            points3D: 3D points, numpy array of shape (N, 3)
        """
        points3D = points3D.reshape(-1, 3)
        points3D = points3D.T
        points_ = self.R @ points3D + self.T.reshape(-1, 1) #输出为3xN（是转换为齐次坐标计算的结果）
        points_ = self.K @ points_
        points2D = points_ / points_[2, :]
        return points2D[:2, :].T # (N, 2)
    
    def draw(self, points, out_img_path=None):
        """ Draw 2D points on image
        Args:
            points: 2D points, numpy array of shape (N, 2)
        """
        img_W = 2*self.cx # 400*400
        img_H = 2*self.cy
        img = np.zeros((img_H, img_W, 3), dtype=np.uint8)

        point_num = points.shape[0]
        outside_img_count = 0
        for point in points:
            x, y = point
            x, y = int(x), int(y)
            if 0 <= x < img_W and 0 <= y < img_H:
                img = cv2.circle(img, (x, y), 5, (0, 255, 0), -1)
            else:
                outside_img_count += 1
                print(f"Point ({x}, {y}) is outside the image")
        
        print(f"Outside image count: {outside_img_count}, while total points: {point_num}")

        if out_img_path:
            cv2.imwrite(out_img_path, img)
        
        if outside_img_count > 0: return False
        return True

    def move_x_axis_plus(self, step):
        self.x += step
        self.T = np.array([self.x, self.y, self.z])
    
    def move_y_axis_plus(self, step):
        self.y += step
        self.T = np.array([self.x, self.y, self.z])

    def move_z_axis_plus(self, step):
        self.z += step
        self.T = np.array([self.x, self.y, self.z])

    def rotate_yaw_plus(self, degree):
        self.yaw += degree
        self.R = self._get_rotation_matrix(self.yaw, self.pitch, self.roll)
    
    def rotate_pitch_plus(self, degree):
        self.pitch += degree
        self.R = self._get_rotation_matrix(self.yaw, self.pitch, self.roll)
    
    def rotate_roll_plus(self, degree):
        self.roll += degree
        self.R = self._get_rotation_matrix(self.yaw, self.pitch, self.roll)

    def reset(self):
        self.x = self.configs["x"]
        self.y = self.configs["y"]
        self.z = self.configs["z"]
        self.T = np.array([self.x, self.y, self.z])

        self.yaw = self.configs["yaw"]
        self.pitch = self.configs["pitch"]
        self.roll = self.configs["roll"]
        self.R = self._get_rotation_matrix(self.yaw, self.pitch, self.roll)

In [101]:
# show K, R, T
# def test1():
#     camera = CameraModel()
#     camera.show_K()
#     camera.show_R()
#     camera.show_T()
# test1()

### 相机模型的使用方法

**主要成员函数如下：**
- `CameraModel.project(points3D)`: 将3D空间中的点投影到相机平面上;
    - 输入
        - `points3D`：为3D空间中的点，类型为`np.ndarray`，尺寸为`(N,3)`，N为点的个数。
    - 返回值
        - `points2D`: 为2D图像平面上的坐标，类型为`np.ndarray`，尺寸为`(N,2)`。
        
- `CameraModel.draw(points2D, out_img_path=None)`: 在图像上绘制投影点，并检验投影点是否在图像内部;
    - 输入：
        - `points2D`：2D平面上的坐标，类型为`np.ndarray`，尺寸为`(N,2)`。
        - `out_img_path`：保存图像的路径，类型为`str`，默认值为`None`，此时不保存图像。
    - 返回值
        - `True`：表示所有投影点都在图像内部
        - `False`：表示有投影点在图像外部

- `CameraModel.move_<axis_name>_plus(step)`：沿着世界坐标系的坐标轴`axis_name`的正方向移动相机;
    - axis_name: `x_axis`, `y_axis`, `z_axis`
    - 输入
        - `step`：移动的步长，类型为`float` （正负均可）。
    - 返回值
        - 无

- `CameraModel.rotate_<angel_name>(degree)`: 改变相机的旋转角，以相机自身坐标系为参照;
    - 相机自身坐标系的x轴指向右，y轴指向下，z轴指向相机的观察方向。
    - angel_name: 旋转角名称 (因为相机坐标系的轴名字定义，与一般的欧拉角定义不同)
        - `yaw`: 绕y轴旋转
        - `pitch`: 绕x轴旋转
        - `roll`: 绕z轴旋转
    - 输入
        - `degree`：旋转的角度，类型为`float` （单位为度，正负均可）。
    - 返回值
        - 无

- `CameraModel.reset()`: 重置相机的位置和旋转角;

**推荐使用流程为：先移动相机，然后进行投影，得到投影点后用`draw`函数检验点的有效性（在图像内），收集有效投影点集合用以标定**


In [102]:
"""3D 棋盘格模型"""
class chessboard3D(object):
    """
    """
    dft_configs = {
        "square_size": 1,
        "rows": 8,
        "cols": 8,
        "z_plane": 0,
    }

    def __init__(self, configs={}) -> None:
        self.configs = {**self.dft_configs, **configs}
        self.square_size = self.configs["square_size"]
        self.rows = self.configs["rows"]
        self.cols = self.configs["cols"]
        self.z_plane = self.configs["z_plane"]
        self.points = self._generate_points()
    
    def _generate_points(self):
        points = []
        for row in range(self.rows):
            for col in range(self.cols):
                points.append([col*self.square_size, row*self.square_size, self.z_plane])
        return np.array(points) # (N, 3)

    def return_points(self):
        return self.points

### 3维棋盘格模型的使用方法

**主要成员函数如下：**
- `ChessboardModel.return_points()`: 获取3D棋盘格模型的角点坐标;
    - 返回值
        - `points3D`：为3D空间中的点，类型为`np.ndarray`，尺寸为`(N,3)`，N为点的个数。

---
## 相机标定的实现

在运行完上述的代码块后，你需要在下面的代码块中实现相机的标定，并用Markdown块完成实验报告。

**实现及报告要求：**

- 相机的标定可以使用opencv库中的函数`cv2.calibrateCamera`，也可以自己实现DLT算法（可酌情加分）。

- OpenCV相机标定可参考：https://docs.opencv.org/3.4/dc/dbb/tutorial_py_calibration.html

- 请输出最后得到的相机内参数，包括焦距、主点坐标。
    
- 开放题回答：
    - 请尝试不同的相机移动方式，观察标定结果的差异，尝试总结相机移动的最佳策略。
    - 尝试不同次数的二维点投影，观察标定结果的差异，尝试总结标定所用的图像数量（即二维投影点集合的数量）的最佳策略。（越多越好？）

**评分标准：**
- 实现相机标定的正确性（15分）
- 相机参数的精度（5分）
- 开放题的回答（5+5分）

---

以下为答题区域，可以使用多个代码块和markdown块。

In [103]:
import random

random.seed(20)

def adjust_parameters():
    # give out random numbers to adjust the parameters
    type_ = random.randint(0, 5)
    if type_ in [0, 1, 2]:
        return type_, random.randint(0, 10)
    elif type_ in [3, 4, 5]:
        return type_, random.uniform(0, 2)

In [104]:
# put the camera in different positions and take pictures, specifically, move the camera to the upper left, upper right, lower left, lower right, and center of the chessboard, and take pictures respectively
def adjust_camera_position(type_, camera_model):
    camera = camera_model
    camera.move_x_axis_plus(-3.5)
    camera.move_y_axis_plus(-3.5)
    # x positive -> right; y positive -> down
    if type_ == 0: # middle
        pass
    elif type_ == 1: # middle plus z
        camera.move_z_axis_plus(5)
    elif type_ == 2: #grad to top left
        camera.move_x_axis_plus(-3.5)
        camera.move_y_axis_plus(-1.5)
        camera.rotate_yaw_plus(45)
        camera.rotate_pitch_plus(45)
    elif type_ == 3: #grad to top left plus z
        camera.move_x_axis_plus(-3.5)
        camera.move_y_axis_plus(-3.5)
        camera.rotate_yaw_plus(45)
        camera.rotate_pitch_plus(45)
        camera.move_z_axis_plus(5)
    elif type_ == 4: #grad to top right
        camera.move_x_axis_plus(3.5)
        camera.move_y_axis_plus(-3.5)
        camera.rotate_yaw_plus(-45)
        camera.rotate_pitch_plus(45)
    elif type_ == 5: #grad to top right plus z
        camera.move_x_axis_plus(3.5)
        camera.move_y_axis_plus(-3.5)
        camera.rotate_yaw_plus(-45)
        camera.rotate_pitch_plus(45)
        camera.move_z_axis_plus(5)
    elif type_ == 6: #grad to bottom left
        camera.move_x_axis_plus(-3.5)
        camera.move_y_axis_plus(3.5)
        camera.rotate_yaw_plus(-45)
        camera.rotate_pitch_plus(-45)
    elif type_ == 7: #grad to bottom left plus z
        camera.move_x_axis_plus(-3.5)
        camera.move_y_axis_plus(3.5)
        camera.rotate_yaw_plus(45)
        camera.rotate_pitch_plus(-45)
        camera.move_z_axis_plus(5)
    elif type_ == 8: #grad to bottom right
        camera.move_x_axis_plus(3.5)
        camera.move_y_axis_plus(3.5)
        camera.rotate_yaw_plus(-45)
        camera.rotate_pitch_plus(-45)
    elif type_ == 9: #grad to bottom right plus z
        camera.move_x_axis_plus(3.5)
        camera.move_y_axis_plus(3.5)
        camera.rotate_yaw_plus(-45)
        camera.rotate_pitch_plus(-45)
        camera.move_z_axis_plus(5)
        

In [105]:
# # test block
# camera = CameraModel()
# chessboard = chessboard3D()
# type_ = 6
# adjust_camera_position(type_, camera)
# points3D = chessboard.return_points()
# points2D = camera.project(points3D)
# camera.draw(points2D, "test_{}.png".format(type_))

Outside image count: 0, while total points: 64


True

In [106]:
valid_points2D = [] # list of numpy arrays
valid_points3D = []

# time start
import time
start = time.time()

run_times = 10
count = 0
random_ = False
if random_ == True:
    while count <= run_times:
        # Create a camera model and a chessboard model
        camera = CameraModel()
        chessboard = chessboard3D()

        # change the parameters times
        for i in range(5):
            type_, value = adjust_parameters()
            if type_ == 0:
                camera.move_x_axis_plus(value)
            elif type_ == 1:
                camera.move_y_axis_plus(value)
            elif type_ == 2:
                camera.move_z_axis_plus(value)
            elif type_ == 3:
                camera.rotate_yaw_plus(value)
            elif type_ == 4:
                camera.rotate_pitch_plus(value)
            elif type_ == 5:
                camera.rotate_roll_plus(value)

        # Get the 3D points from the chessboard model
        points3D = chessboard.return_points() # (N, 3)

        # Project the 3D points onto the 2D image plane using the camera model
        points2D = camera.project(points3D) # (N, 2)

        # Draw the 2D points on an image and check if they are inside the image boundaries
        valid_projection = camera.draw(points2D)

        # Collect points only if the projection is valid
        if valid_projection:
            count += 1
            valid_points2D.append(points2D)
            valid_points3D.append(points3D)
else:
    for i in range(0,9):
        # Create a camera model and a chessboard model
        camera = CameraModel()
        chessboard = chessboard3D()

        # change the parameters
        adjust_camera_position(i, camera)

        # Get the 3D points from the chessboard model
        points3D = chessboard.return_points() # (N, 3)

        # Project the 3D points onto the 2D image plane using the camera model
        points2D = camera.project(points3D) # (N, 2)

        # Draw the 2D points on an image and check if they are inside the image boundaries
        valid_projection = camera.draw(points2D)

        # Collect points only if the projection is valid
        if valid_projection:
            count += 1
            valid_points2D.append(points2D)
            valid_points3D.append(points3D)

Outside image count: 0, while total points: 64
Outside image count: 0, while total points: 64
Outside image count: 0, while total points: 64
Outside image count: 0, while total points: 64
Outside image count: 0, while total points: 64
Outside image count: 0, while total points: 64
Outside image count: 0, while total points: 64
Outside image count: 0, while total points: 64
Outside image count: 0, while total points: 64


In [107]:
# print("valid_points2D: ", valid_points2D)
# print("valid_points3D: ", valid_points3D)
# print("valid_points2D_shape: ", np.array(valid_points2D).shape) # list of numpy array
# print("valid_points3D_shape: ", np.array(valid_points3D).shape)

In [108]:
# pay attention to the shape and data type of the objectPoints and imagePoints
objectPoints = [np.array(points.reshape(-1, 1, 3),dtype=np.float32) for points in valid_points3D] # (imgNum, 64, 1, 3)
imagePoints = [np.array(points.reshape(-1, 1, 2),dtype=np.float32) for points in valid_points2D] # (imgNum, 64, 1, 2)

In [109]:
# calculate the camera matrix
ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objectPoints, imagePoints, (400, 400), None, None)

camera.show_K()

# mtx is the intrinsic parameters
print("Camera matrix:\n", mtx)

K: [[200   0 200]
 [  0 200 200]
 [  0   0   1]]
Camera matrix:
 [[200.00000851   0.         199.99999857]
 [  0.         200.00000714 199.99999161]
 [  0.           0.           1.        ]]


In [110]:
# calculate the difference
diff = sum(sum(np.abs(mtx - camera.K)))
print("Difference :\n", diff)
# time end
end = time.time()
print("Time: ", end - start)
# imgNum : difference ；time
# 1 : 0.056010101876495355 :0.08255124092102051
# 5 : 0.019085119902143788 : 0.09501981735229492
# 10 : 0.00553201687012006 : 0.21959805488586426
# 20: 0.006914235597633933 : 0.8205409049987793
# 50 : 0.0020809845348708222 : 15.410118579864502
# 100 : 0.0009327795618503387 : 64.56432819366455


Difference :
 2.5465785085998505e-05
Time:  0.11002564430236816


---
#### 1.尝试不同的相机移动方式，观察标定结果的差异，尝试总结相机移动的最佳策略。

#### Ans:
**移动相机的最佳策略应该确保从多个不同的位置和角度获取图像（对应点），实现相机位置与角度的多样性。这种多样性从多个方面对相机的标定产生影响:**
- 不同距离（改变z）：从不同距离下进行图像投影，获得多种尺度的图片，有利于更好的估计焦距。
- 覆盖视场范围：在不同图像中覆盖相机视场的不同部分，也即是在多个不同的机位获取图片投影，有助于径向畸变的感知（此次任务是理想相机，不涉及畸变）。
- 旋转相机角度：当相机有较严重的镜头畸变时，获取沿不同轴旋转的图像，有助于更准确的估计相机内部参数。
- 避免重复：在单一位置、单一角度获取多张图片没有太大作用，需要确保不同角度的平衡覆盖。

**总结**：最好的相机移动策略是最大限度地覆盖相机的视场，包括多种距离和角度，并确保校准模式在图像中的不同方向和尺度的良好表示。这种方法为准确估计相机的内部参数提供了丰富的数据集。

**实验：**
- 只提供相机角度变化不改变距离：此时，虽然只有5张标定图片提供信息，但标定矩阵与真实值之间的误差只有4.408e-05，几乎可以忽略不计，说明图片的方位信息对图片标定很重要。当然，虽然没有显式地提供距离的变化，但是因为有了旋转的角度，标定板不同位置与相机平面的距离不是恒定的，这也在一定程度上提供了距离的信息。另外，由于五张图片提供的空间信息几乎没有重复，所以标定效果显著好于随机改变位置提供五张标定图片的效果（difference=0.019），且用时更少。
    | imgNum | difference | time |
    |--------|------------|------|
    | 5      |  4.408e-05 | 0.078|
- 只改变距离：只进行图片距离的变换，也就是将相机放置在标定物中心，只改变相机与标定物之间的垂直距离(z坐标)，不改变x,y坐标与角度。此时,相机的内参数被严重错误的估计，因为缺少了z轴方向移动距离的信息，仅由距离改变的提供的信息不能唯一的解出相机矩阵。由此可见，距离的改变是必要但不充分的。
    | imgNum | difference | time |
    |--------|------------|------|
    | 2      |  4.408e-05 | 0.078|

----------


#### 2.尝试不同次数的二维点投影，观察标定结果的差异，尝试总结标定所用的图像数量（即二维投影点集合的数量）的最佳策略

#### Ans:
**在相机标定过程中，标定的最佳策略是保证足够的多样性和覆盖范围，而不仅仅是追求数量。**
虽然更多的图像可以提供更多的数据点，但是使用过多的重复图像可能会造成过拟合，使得标定的参数过度适应于提供的数据集，而在更general的应用中反而不是最优的。
- 理想数量：在图片数量较小时（2-3张），获得的数据通常不能提供足够的视角和位置信息，在这种情况下，增加不同角度的图片数量，以提供更多的图片信息，往往可以提高图片准确性。另外，精心挑选的条件下，10到20张图像就能提供足够的数据实现较好的标定效果。这些图片足以覆盖标定物。此时再增加图片数量，计算量快速上升，却不会提供明显的增益。

**总结**：要仔细选择和评估每张用于标定的图像，以确保它们为标定过程提供有价值的信息。

**实验：**
- 随机改变相机位置：在该实验中，通过设置随机数，随机生成改变的相机位置类型，其中类型整数0，1，2分别代表更改相机的x,y,z轴坐标，变化量为0-10之间的随机整数；类型整数3，4，5分别改变相机绕x,y,z轴角度，变化量为0-2度之间的均匀分布的实数。在该条件下，分别获取1，5，10，20，50，100张有效图片（每个点都在摄影平面内），统计相机内参矩阵真实值与估计值之间差距，结果如下：
    | imgNum | difference | time |
    |--------|------------|------|
    | 1      | 0.056      | 0.082|
    | 5      | 0.019      | 0.095|
    | 10     | 0.005      | 0.219|
    | 20     | 0.006      | 0.820|
    | 50     | 0.002      | 15.410|
    | 100    | 0.0009     | 64.564|
    
    可以观察到随着有效图片数量的增加，矩阵真实值与估计值之间的差距呈逐渐减小的趋势，消耗的时间也在逐渐上升。但是，我们可以观察到，当图片数量达到10-20张时，所获得的估计值已经足够接近真实值，另外，当图片数量从20变为50（增加1.5倍）时，消耗的时间从0.820上升到15.410（增加17.8倍），时间复杂度非线性。所以，理想情况下，10-20张图片足够提高较好的标定效果与较低的计算开销。
- 设计相机位置：与随机改变相机位置不同，在该实验中，我对相机位置进行了专门的设计。首先，我将相机放在中心、左上、右上、左下、右下五个位置，以提供不同视角与位置的图片信息，并且保证了每个方向的平衡覆盖（每个方向图片数量均衡）；另外，我在五个位置上再施加了z轴的变化，拉远相机与标定物的距离，获得不同尺度的图片，更有利于相机焦距的标定。在该实验条件下所得结果如下：
    | imgNum | difference | time |
    |--------|------------|------|
    | 10      |  2.954e-05 | 0.155|

    此时，只有10张图片但提供了比随机条件下100张有效图片更精确的标定，标定误差达到1e-5的级别，几乎可以忽略。该实验进一步说明标定的最佳策略是保证足够的多样性和覆盖范围，而不仅仅是追求数量。
---